In [1]:
import numpy as np
import pandas as pd
import os

from utils import evaluate_algorithm, split_interactions

In [3]:
def get_data(create_new=False):
    
    if create_new:
        path = 'MRS_Challenge_2022_data'

        usr_path = os.path.join(path, 'MRSC_2022_demo.txt')
        itm_path = os.path.join(path, 'MRSC_2022_tracks.txt')
        inter_path = os.path.join(path, 'MRS_Challenge_2022_data/MRSC_2022_inter.txt')

        train_inter_files = []
        test_inter_files = []

        num_splits = 2
        p_i = 0.3
        p_u = 0.5

        user_file_path = None
        inter_file_path = None

        user_file_path = usr_path
        inter_file_path = inter_path

        for i in range(num_splits):

            split_interactions(inter_file=inter_file_path,
                               user_file_path = user_file_path,
                               p_u = p_u,
                               p_i = p_i,
                               res_test_file="train_test_data/inter_TEST_" + str(i) + ".txt",
                               res_train_file="train_test_data/inter_TRAIN_" + str(i) + ".txt")

            train_inter_files.append("train_test_data/inter_TRAIN_" + str(i) + ".txt")
            test_inter_files.append("train_test_data/inter_TEST_" + str(i) + ".txt")

In [4]:
train_inter_files = ['train_test_data/inter_TRAIN_0.txt', 'train_test_data/inter_TRAIN_1.txt']
test_inter_files = ['train_test_data/inter_TEST_0.txt', 'train_test_data/inter_TEST_0.txt']
path = 'MRS_Challenge_2022_data'

config = {
        "algorithm": None,
        "inter_train_file_paths": train_inter_files,
        "inter_test_file_paths": test_inter_files,
        "user_file_path": os.path.join(path, 'MRSC_2022_demo.txt'),
        "item_file_path": os.path.join(path, 'MRSC_2022_tracks.txt'),
        "top_k": 15,
        "n": 0,
        "f": 0
    }

class POP:
    def __init__(self, inter_matrix, df_users, top_k=15):
        self.inter_matrix = inter_matrix
        self.top_k = top_k
        self.df_users = df_users
    
    def predict(self, user):
        if type(user) == int:
            item_pop = self.inter_matrix.sum(axis=0)
            items_seen = np.nonzero(self.inter_matrix[user])
            item_pop[items_seen] = 0
            rec = (-item_pop).argsort()[:self.top_k]
            
        else:
            for u in user:
                rec = np.full((len(self.df_users), config['top_k']), -1)

                item_pop = self.inter_matrix.sum(axis=0)
                items_seen = np.nonzero(self.inter_matrix[u])
                item_pop[items_seen] = 0
                top_pop = (-item_pop).argsort()[:self.top_k]

                rec[u] = top_pop

        return rec

In [5]:
evaluate_algorithm(config, POP)

/home/liondungl/.programs/miniconda3/envs/deeplearning/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/liondungl/.programs/miniconda3/envs/deeplearning/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'m': {'ndcg': 0.0}, 'f': {'ndcg': nan}, 'all': {'ndcg': 0.0}}